In [2]:
import pandas as pd

In [3]:
products = pd.read_csv('../raw/products.csv')
categories = pd.read_csv('../raw/categories.csv')
inventory = pd.read_csv("../raw/inventory.csv")
warehouses = pd.read_csv("../raw/warehouses.csv")
order_items = pd.read_csv("../raw/order_items.csv")

In [4]:
print(products.shape)
print(categories.shape)
print(inventory.shape)
print(warehouses.shape)
print(order_items.shape)

(150, 10)
(12, 3)
(300, 5)
(2, 5)
(964, 7)


In [5]:
inventory_w = pd.merge(inventory, warehouses, on = "warehouse_id", how = "left", suffixes=['_inventory','_warehouses'])
print(inventory_w.shape)

(300, 9)


In [6]:
product_c = pd.merge(products, categories, on = "category_id", how = "left", suffixes=['_products','_categories'])
print(product_c.shape)

(150, 12)


In [7]:
product_2 = pd.merge(product_c, inventory_w, on = "product_id", how = "left")
print(product_2.shape)

(300, 20)


In [8]:
order_items_2 = pd.merge(order_items, product_2, on = "product_id", how = "left", suffixes=['_order_items',''])
print(order_items_2.shape)

(1928, 26)


In [9]:
new_order_items = order_items_2.copy()

In [10]:
orders = pd.read_csv("../raw/orders.csv")
payments = pd.read_csv("../raw/payments.csv")
shipments = pd.read_csv("../raw/shipments.csv")

In [11]:
print(orders.shape)
print(payments.shape)
print(shipments.shape)

(400, 14)
(399, 8)
(209, 8)


In [12]:
payments.order_id.value_counts().sum()

np.int64(399)

In [13]:
orders_p = pd.merge(orders, payments, on = "order_id", how = "left", suffixes=['_orders','_payments'])
print(orders_p.shape)

(466, 21)


In [14]:
orders_p_s = pd.merge(orders_p, shipments, on = "order_id", how = "left", suffixes=['','_shipments'])
print(orders_p_s.shape)

(466, 28)


In [15]:
customers = pd.read_csv("../raw/customers.csv")
customer_addresses = pd.read_csv("../raw/customer_addresses.csv")

In [16]:
print(customers.shape)
print(customer_addresses.shape)

(250, 9)
(394, 10)


In [17]:
orders_p_s_c = pd.merge(orders_p_s, customers, on = "customer_id", how = "left", suffixes=['','_customers'])
print(orders_p_s_c.shape)

(466, 36)


In [18]:
orders_p_s_c_ba = pd.merge(orders_p_s_c, customer_addresses.add_suffix('_billing'), left_on = "billing_address_id", right_on="address_id_billing", how = "left")
print(orders_p_s_c_ba.shape)

(466, 46)


In [19]:
orders_p_s_c_ba_sa = pd.merge(orders_p_s_c_ba, customer_addresses.add_suffix('_shipping'), left_on = "shipping_address_id", right_on="address_id_shipping", how = "left")
print(orders_p_s_c_ba_sa.shape)

(466, 56)


In [20]:
new_orders = orders_p_s_c_ba_sa.copy()

In [21]:
channels = pd.read_csv("../raw/channels.csv")
order_marketing = pd.read_csv("../raw/order_marketing.csv")
campaigns = pd.read_csv("../raw/campaigns.csv")

In [22]:
print(channels.shape)
print(order_marketing.shape)
print(campaigns.shape)

(4, 3)
(400, 8)
(12, 9)


In [23]:
new_orders_c = pd.merge(new_orders, channels, on = "channel_id", how = "left", suffixes=['','_channels']) 
print(new_orders_c.shape)

(466, 58)


In [24]:
order_marketing_c = pd.merge(order_marketing.add_suffix('_order_marketing'), channels, left_on ="channel_id_order_marketing", right_on = 'channel_id', how = "left", suffixes=['','_channels'])
print(order_marketing_c.shape)

(400, 11)


In [25]:
order_marketing_c_c = pd.merge(order_marketing_c, campaigns.add_suffix('_campaigns'), left_on ="campaign_id_order_marketing", right_on ="campaign_id_campaigns", how = "left")
print(order_marketing_c_c.shape)

(400, 20)


In [26]:
order_marketing_c_c.columns.to_list()

['order_id_order_marketing',
 'channel_id_order_marketing',
 'campaign_id_order_marketing',
 'utm_source_order_marketing',
 'utm_medium_order_marketing',
 'utm_campaign_order_marketing',
 'utm_content_order_marketing',
 'utm_term_order_marketing',
 'channel_id',
 'channel_name',
 'description',
 'campaign_id_campaigns',
 'channel_id_campaigns',
 'campaign_name_campaigns',
 'utm_source_campaigns',
 'utm_medium_campaigns',
 'utm_campaign_campaigns',
 'start_date_campaigns',
 'end_date_campaigns',
 'budget_ars_campaigns']

In [27]:
new_orders_c_2 = pd.merge(new_orders_c, order_marketing_c_c, left_on = "order_id", right_on = "order_id_order_marketing", how = "left", suffixes=['_orders',''])
print(new_orders_c_2.shape)

(466, 78)


In [28]:
one_big_table = pd.merge(new_order_items, new_orders_c_2, on = "order_id", how = "left")
print(one_big_table.shape)

(2224, 103)


In [29]:
# one_big_table.columns.to_list()
id_columns_to_drop = [x for x in one_big_table.columns.to_list() if '_id' in x][1:]
one_big_table = one_big_table.drop(columns=id_columns_to_drop)
print(one_big_table.shape)

(2224, 81)


In [30]:
one_big_table = one_big_table.reset_index().rename(columns={'index':'id'})
one_big_table['id'] = one_big_table['id'] + 1
one_big_table.head()

,id,order_id,line_number,quantity,unit_price_order_items,discount_amount,tax_amount,sku,product_name,brand,...,utm_term_order_marketing,channel_name,description,campaign_name_campaigns,utm_source_campaigns,utm_medium_campaigns,utm_campaign_campaigns,start_date_campaigns,end_date_campaigns,budget_ars_campaigns
0,1,100001,1,2,97309.59,13623.34,38009.13,SKU-00137,Norte Mochilas 137,Norte,...,mochilas,Web,Tienda online desktop/mobile,Retargeting Julio 2025 - Web,meta,social,retargeting_jul_2025,2025-08-08,2025-08-27,3157411.81
1,2,100001,1,2,97309.59,13623.34,38009.13,SKU-00137,Norte Mochilas 137,Norte,...,mochilas,Web,Tienda online desktop/mobile,Retargeting Julio 2025 - Web,meta,social,retargeting_jul_2025,2025-08-08,2025-08-27,3157411.81
2,3,100001,2,2,69290.55,2771.62,28519.99,SKU-00148,Norte Mochilas 148,Norte,...,mochilas,Web,Tienda online desktop/mobile,Retargeting Julio 2025 - Web,meta,social,retargeting_jul_2025,2025-08-08,2025-08-27,3157411.81
3,4,100001,2,2,69290.55,2771.62,28519.99,SKU-00148,Norte Mochilas 148,Norte,...,mochilas,Web,Tienda online desktop/mobile,Retargeting Julio 2025 - Web,meta,social,retargeting_jul_2025,2025-08-08,2025-08-27,3157411.81
4,5,100001,3,3,55916.19,8387.43,33465.84,SKU-00053,Estilo Remeras 053,Estilo,...,mochilas,Web,Tienda online desktop/mobile,Retargeting Julio 2025 - Web,meta,social,retargeting_jul_2025,2025-08-08,2025-08-27,3157411.81


In [31]:
one_big_table.to_csv('../denormalised tables/one_big_table.csv')